In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import haversine as hs
from haversine import Unit
import numpy as np

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
# Load the housing sale prices data.
houses_df = pd.read_csv('sale_price.csv')

# Add columns for assigned elementary, middle and high schools.
houses_df['elem'] = np.nan
houses_df['middle'] = np.nan
houses_df['high'] = np.nan

houses_df.head()

,Unnamed: 0,zpid,zipcode,city,state,latitude,longitude,price,dateSold,bathrooms,...,isPremierBuilder,isZillowOwned,currency,country,taxAssessedValue,lotAreaValue,lotAreaUnit,elem,middle,high
0,0,37151520,20854,Potomac,MD,39.043986,-77.256456,800000.0,1625641200000,4.0,...,False,False,USD,USA,1081300.0,5.10,acres,NaN,NaN,NaN
1,1,246554034,20814,Bethesda,MD,39.022005,-77.107586,800000.0,1617606000000,1.0,...,False,False,USD,USA,507900.0,0.25,acres,NaN,NaN,NaN
2,2,37181535,20816,Bethesda,MD,38.958795,-77.111890,800000.0,1624604400000,3.0,...,False,False,USD,USA,721567.0,8750.00,sqft,NaN,NaN,NaN
3,3,37182564,20817,Bethesda,MD,38.974071,-77.131494,800000.0,1604563200000,3.0,...,False,False,USD,USA,697767.0,9148.00,sqft,NaN,NaN,NaN
4,4,37096069,20854,Potomac,MD,39.041634,-77.172128,800000.0,1620370800000,3.0,...,False,False,USD,USA,611000.0,9811.00,sqft,NaN,NaN,NaN


In [3]:
# Utility function to return a list of clean school names.
def get_schools(schools_df):
    
    ret = []
    
    # Access all schools and put them in a list.
    for s in schools_df['School'].tolist():
        
        # Split the string.
        arr = s.split()
        
        # If the last word is 'School', drop it.
        if arr[len(arr)-1].lower() == 'school':
            arr.pop()
            
        # Convert all words to lowercase.
        for i in range(0, len(arr)):
            arr[i] = arr[i].lower()
            
        ret.append(' '.join(arr))
    
    return ret

In [4]:
# Utility function to return a "cleaned school" name.
def clean_school_name(s):
    
    arr = s.split()
    
    # If the last word is 'School', drop it.
    if arr[len(arr)-1].lower() == 'school':
        arr.pop()
        
    # Convert all words to lowercase.
    for i in range(0, len(arr)):
        arr[i] = arr[i].lower()
        
    return ' '.join(arr)

In [5]:
# Load the school ratings data.
schools_df = pd.read_csv('mcps_school_ratings.csv')

# Get a list of "cleaned" elementary school names.
elem_schools = get_schools(schools_df.loc[schools_df['Type'] == 'E'])

# Get a list of "cleaned" middle school names.
middle_schools = get_schools(schools_df.loc[schools_df['Type'] == 'M'])

# Get a list of "cleaned" high school names.
high_schools = get_schools(schools_df.loc[schools_df['Type'] == 'H'])

In [6]:
# Utility function to assign primary school to the given address based on geographical proximity.
def get_assigned_primary_school(gmap_results, loc):
    
    elem_distance = 100000.0
    assigned_elem = None

    for s in gmap_results['results']:
                
        s_name = clean_school_name(s['name'])
        s_lat = s['geometry']['location']['lat']
        s_lng = s['geometry']['location']['lng']

    
        # Check if this name matches to a potential elementary school name in MCPS.
        if s_name in elem_schools:
        
            dist = hs.haversine(loc,
                                (s_lat, s_lng),
                                unit=Unit.MILES)
        
            if dist < elem_distance:
                elem_distance = dist
                assigned_elem = s_name
                #print (f'Found elementary school {assigned_elem} with dist {elem_distance}')
    
    return assigned_elem

In [7]:
# Utility function to assign middle and high schools to the given address based on geographical proximity.
def get_assigned_secondary_schools(gmap_results, loc):
    
    middle_distance = 100000.0
    high_distance = 100000.0
    
    assigned_middle = None
    assigned_high = None

    for s in gmap_results['results']:
            
        s_name = clean_school_name(s['name'])
        s_lat = s['geometry']['location']['lat']
        s_lng = s['geometry']['location']['lng']

    
        # Check if this name matches to a potential middle school name in MCPS.
        if s_name in middle_schools:
        
            dist = hs.haversine(loc,
                                (s_lat, s_lng),
                                unit=Unit.MILES)
        
            if dist < middle_distance:
                middle_distance = dist
                assigned_middle = s_name
                #print (f'Found middle school {assigned_middle} with dist {middle_distance}')
            
        # Check if this name matches to a potential high school name in MCPS.
        elif s_name in high_schools:
        
            dist = hs.haversine(loc,
                               (s_lat, s_lng),
                                unit=Unit.MILES)
        
            if dist < high_distance:
                high_distance = dist
                assigned_high = s_name
                #print (f'Found high school {assigned_high} with dist {high_distance}')
    
    return (assigned_middle, assigned_high)

In [10]:
####################### Assign Elementary School. #############################################
###### IMPORTANT: Google API may ratelimit, so it may be necessary to kill the process. ######
###### If this happens, export the csv and start from the next index onwards. ################
###### Once ran, do NOT run this code again causing thousands of gmap API calls. #############

# Set parameters to search for primary schools within 10000 meters.
params = {
    "radius": 10000,
    "type": "primary_school",
    "key": gkey
}

# Construct the baseurl.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in houses_df.iterrows():
    
    # Add lat, lng to the parameters.
    params['location'] = f"{row['latitude']},{row['longitude']}"
    
    # Make request and retrieve the JSON data from the search. 
    #try:
    #    potential_elem_schools = requests.get(base_url, params=params).json()
    #    assigned_elem_school = get_assigned_primary_school(potential_elem_schools, 
    #                                                      (row['latitude'], row['longitude'])
    #                                                      )
        
        # Assign school.
    #    if assigned_elem_school:
    #        houses_df.loc[index, 'elem'] = assigned_elem_school
    
    #except Exception as e:
    #    print (f"Index: {index}, Zpid: {row['zpid']}, response message {schools['message']}")
        
    print(f'Index: {index}')

Index: 5420
Index: 5421
Index: 5422
Index: 5423
Index: 5424
Index: 5425
Index: 5426
Index: 5427
Index: 5428
Index: 5429
Index: 5430
Index: 5431
Index: 5432
Index: 5433
Index: 5434
Index: 5435
Index: 5436
Index: 5437
Index: 5438
Index: 5439
Index: 5440
Index: 5441
Index: 5442
Index: 5443
Index: 5444
Index: 5445
Index: 5446
Index: 5447
Index: 5448
Index: 5449
Index: 5450
Index: 5451
Index: 5452
Index: 5453
Index: 5454
Index: 5455
Index: 5456
Index: 5457
Index: 5458
Index: 5459
Index: 5460
Index: 5461
Index: 5462
Index: 5463
Index: 5464
Index: 5465
Index: 5466
Index: 5467
Index: 5468
Index: 5469
Index: 5470
Index: 5471
Index: 5472
Index: 5473
Index: 5474
Index: 5475
Index: 5476
Index: 5477
Index: 5478
Index: 5479
Index: 5480
Index: 5481
Index: 5482
Index: 5483
Index: 5484
Index: 5485
Index: 5486
Index: 5487
Index: 5488
Index: 5489
Index: 5490
Index: 5491
Index: 5492
Index: 5493
Index: 5494
Index: 5495
Index: 5496
Index: 5497
Index: 5498
Index: 5499
Index: 5500
Index: 5501
Index: 5502
Inde

Index: 6103
Index: 6104
Index: 6105
Index: 6106
Index: 6107
Index: 6108
Index: 6109
Index: 6110
Index: 6111
Index: 6112
Index: 6113
Index: 6114
Index: 6115
Index: 6116
Index: 6117
Index: 6118
Index: 6119
Index: 6120
Index: 6121
Index: 6122
Index: 6123
Index: 6124
Index: 6125
Index: 6126
Index: 6127
Index: 6128
Index: 6129
Index: 6130
Index: 6131
Index: 6132
Index: 6133
Index: 6134
Index: 6135
Index: 6136
Index: 6137
Index: 6138
Index: 6139
Index: 6140
Index: 6141
Index: 6142
Index: 6143
Index: 6144
Index: 6145
Index: 6146
Index: 6147
Index: 6148
Index: 6149
Index: 6150
Index: 6151
Index: 6152
Index: 6153
Index: 6154
Index: 6155
Index: 6156
Index: 6157
Index: 6158
Index: 6159
Index: 6160
Index: 6161
Index: 6162
Index: 6163
Index: 6164
Index: 6165
Index: 6166
Index: 6167
Index: 6168
Index: 6169
Index: 6170
Index: 6171
Index: 6172
Index: 6173
Index: 6174
Index: 6175
Index: 6176
Index: 6177
Index: 6178
Index: 6179
Index: 6180
Index: 6181
Index: 6182
Index: 6183
Index: 6184
Index: 6185
Inde

Index: 6787
Index: 6788
Index: 6789
Index: 6790
Index: 6791
Index: 6792
Index: 6793
Index: 6794
Index: 6795
Index: 6796
Index: 6797
Index: 6798
Index: 6799
Index: 6800
Index: 6801
Index: 6802
Index: 6803
Index: 6804
Index: 6805
Index: 6806
Index: 6807
Index: 6808
Index: 6809
Index: 6810
Index: 6811
Index: 6812
Index: 6813
Index: 6814
Index: 6815
Index: 6816
Index: 6817
Index: 6818
Index: 6819
Index: 6820
Index: 6821
Index: 6822
Index: 6823
Index: 6824
Index: 6825
Index: 6826
Index: 6827
Index: 6828
Index: 6829
Index: 6830
Index: 6831
Index: 6832
Index: 6833
Index: 6834
Index: 6835
Index: 6836
Index: 6837
Index: 6838
Index: 6839
Index: 6840
Index: 6841
Index: 6842
Index: 6843
Index: 6844
Index: 6845
Index: 6846
Index: 6847
Index: 6848
Index: 6849
Index: 6850
Index: 6851
Index: 6852
Index: 6853
Index: 6854
Index: 6855
Index: 6856
Index: 6857
Index: 6858
Index: 6859
Index: 6860
Index: 6861
Index: 6862
Index: 6863
Index: 6864
Index: 6865
Index: 6866
Index: 6867
Index: 6868
Index: 6869
Inde

Index: 7470
Index: 7471
Index: 7472
Index: 7473
Index: 7474
Index: 7475
Index: 7476
Index: 7477
Index: 7478
Index: 7479
Index: 7480
Index: 7481
Index: 7482
Index: 7483
Index: 7484
Index: 7485
Index: 7486
Index: 7487
Index: 7488
Index: 7489
Index: 7490
Index: 7491
Index: 7492
Index: 7493
Index: 7494
Index: 7495
Index: 7496
Index: 7497
Index: 7498
Index: 7499
Index: 7500
Index: 7501
Index: 7502
Index: 7503
Index: 7504
Index: 7505
Index: 7506
Index: 7507
Index: 7508
Index: 7509
Index: 7510
Index: 7511
Index: 7512
Index: 7513
Index: 7514
Index: 7515
Index: 7516
Index: 7517
Index: 7518
Index: 7519
Index: 7520
Index: 7521
Index: 7522
Index: 7523
Index: 7524
Index: 7525
Index: 7526
Index: 7527
Index: 7528
Index: 7529
Index: 7530
Index: 7531
Index: 7532
Index: 7533
Index: 7534
Index: 7535
Index: 7536
Index: 7537
Index: 7538
Index: 7539
Index: 7540
Index: 7541
Index: 7542
Index: 7543
Index: 7544
Index: 7545
Index: 7546
Index: 7547
Index: 7548
Index: 7549
Index: 7550
Index: 7551
Index: 7552
Inde

Index: 8153
Index: 8154
Index: 8155
Index: 8156
Index: 8157
Index: 8158
Index: 8159
Index: 8160
Index: 8161
Index: 8162
Index: 8163
Index: 8164
Index: 8165
Index: 8166
Index: 8167
Index: 8168
Index: 8169
Index: 8170
Index: 8171
Index: 8172
Index: 8173
Index: 8174
Index: 8175
Index: 8176
Index: 8177
Index: 8178
Index: 8179
Index: 8180
Index: 8181
Index: 8182
Index: 8183
Index: 8184
Index: 8185
Index: 8186
Index: 8187
Index: 8188
Index: 8189
Index: 8190
Index: 8191
Index: 8192
Index: 8193
Index: 8194
Index: 8195
Index: 8196
Index: 8197
Index: 8198
Index: 8199
Index: 8200
Index: 8201
Index: 8202
Index: 8203
Index: 8204
Index: 8205
Index: 8206
Index: 8207
Index: 8208
Index: 8209
Index: 8210
Index: 8211
Index: 8212
Index: 8213
Index: 8214
Index: 8215
Index: 8216
Index: 8217
Index: 8218
Index: 8219
Index: 8220
Index: 8221
Index: 8222
Index: 8223
Index: 8224
Index: 8225
Index: 8226
Index: 8227
Index: 8228
Index: 8229
Index: 8230
Index: 8231
Index: 8232
Index: 8233
Index: 8234
Index: 8235
Inde

...

Index: 12713
Index: 12714
Index: 12715
Index: 12716
Index: 12717
Index: 12718
Index: 12719
Index: 12720
Index: 12721
Index: 12722
Index: 12723
Index: 12724
Index: 12725
Index: 12726
Index: 12727
Index: 12728
Index: 12729
Index: 12730
Index: 12731
Index: 12732
Index: 12733
Index: 12734
Index: 12735
Index: 12736
Index: 12737
Index: 12738
Index: 12739
Index: 12740
Index: 12741
Index: 12742
Index: 12743
Index: 12744
Index: 12745
Index: 12746
Index: 12747
Index: 12748
Index: 12749
Index: 12750
Index: 12751
Index: 12752
Index: 12753
Index: 12754
Index: 12755
Index: 12756
Index: 12757
Index: 12758
Index: 12759
Index: 12760
Index: 12761
Index: 12762
Index: 12763
Index: 12764
Index: 12765
Index: 12766
Index: 12767
Index: 12768
Index: 12769
Index: 12770
Index: 12771
Index: 12772
Index: 12773
Index: 12774
Index: 12775
Index: 12776
Index: 12777
Index: 12778
Index: 12779
Index: 12780
Index: 12781
Index: 12782
Index: 12783
Index: 12784
Index: 12785
Index: 12786
Index: 12787
Index: 12788
Index: 12789

Index: 13344
Index: 13345
Index: 13346
Index: 13347
Index: 13348
Index: 13349
Index: 13350
Index: 13351
Index: 13352
Index: 13353
Index: 13354
Index: 13355
Index: 13356
Index: 13357
Index: 13358
Index: 13359
Index: 13360
Index: 13361
Index: 13362
Index: 13363
Index: 13364
Index: 13365
Index: 13366
Index: 13367
Index: 13368
Index: 13369
Index: 13370
Index: 13371
Index: 13372
Index: 13373
Index: 13374
Index: 13375
Index: 13376
Index: 13377
Index: 13378
Index: 13379
Index: 13380
Index: 13381
Index: 13382
Index: 13383
Index: 13384
Index: 13385
Index: 13386
Index: 13387
Index: 13388
Index: 13389
Index: 13390
Index: 13391
Index: 13392
Index: 13393
Index: 13394
Index: 13395
Index: 13396
Index: 13397
Index: 13398
Index: 13399
Index: 13400
Index: 13401
Index: 13402
Index: 13403
Index: 13404
Index: 13405
Index: 13406
Index: 13407
Index: 13408
Index: 13409
Index: 13410
Index: 13411
Index: 13412
Index: 13413
Index: 13414
Index: 13415
Index: 13416
Index: 13417
Index: 13418
Index: 13419
Index: 13420

Index: 13975
Index: 13976
Index: 13977
Index: 13978
Index: 13979
Index: 13980
Index: 13981
Index: 13982
Index: 13983
Index: 13984
Index: 13985
Index: 13986
Index: 13987
Index: 13988
Index: 13989
Index: 13990
Index: 13991
Index: 13992
Index: 13993
Index: 13994
Index: 13995
Index: 13996
Index: 13997
Index: 13998
Index: 13999
Index: 14000
Index: 14001
Index: 14002
Index: 14003
Index: 14004
Index: 14005
Index: 14006
Index: 14007
Index: 14008
Index: 14009
Index: 14010
Index: 14011
Index: 14012
Index: 14013
Index: 14014
Index: 14015
Index: 14016
Index: 14017
Index: 14018
Index: 14019
Index: 14020
Index: 14021
Index: 14022
Index: 14023
Index: 14024
Index: 14025
Index: 14026
Index: 14027
Index: 14028
Index: 14029
Index: 14030
Index: 14031
Index: 14032
Index: 14033
Index: 14034
Index: 14035
Index: 14036
Index: 14037
Index: 14038
Index: 14039
Index: 14040
Index: 14041
Index: 14042
Index: 14043
Index: 14044
Index: 14045
Index: 14046
Index: 14047
Index: 14048
Index: 14049
Index: 14050
Index: 14051

Index: 14606
Index: 14607
Index: 14608
Index: 14609
Index: 14610
Index: 14611
Index: 14612
Index: 14613
Index: 14614
Index: 14615
Index: 14616
Index: 14617
Index: 14618
Index: 14619
Index: 14620
Index: 14621
Index: 14622
Index: 14623
Index: 14624
Index: 14625
Index: 14626
Index: 14627
Index: 14628
Index: 14629
Index: 14630
Index: 14631
Index: 14632
Index: 14633
Index: 14634
Index: 14635
Index: 14636
Index: 14637
Index: 14638
Index: 14639
Index: 14640
Index: 14641
Index: 14642
Index: 14643
Index: 14644
Index: 14645
Index: 14646
Index: 14647
Index: 14648
Index: 14649
Index: 14650
Index: 14651
Index: 14652
Index: 14653
Index: 14654
Index: 14655
Index: 14656
Index: 14657
Index: 14658
Index: 14659
Index: 14660
Index: 14661
Index: 14662
Index: 14663
Index: 14664
Index: 14665
Index: 14666
Index: 14667
Index: 14668
Index: 14669
Index: 14670
Index: 14671
Index: 14672
Index: 14673
Index: 14674
Index: 14675
Index: 14676
Index: 14677
Index: 14678
Index: 14679
Index: 14680
Index: 14681
Index: 14682

Index: 15237
Index: 15238
Index: 15239
Index: 15240
Index: 15241
Index: 15242
Index: 15243
Index: 15244
Index: 15245
Index: 15246
Index: 15247
Index: 15248
Index: 15249
Index: 15250
Index: 15251
Index: 15252
Index: 15253
Index: 15254
Index: 15255
Index: 15256
Index: 15257
Index: 15258
Index: 15259
Index: 15260
Index: 15261
Index: 15262
Index: 15263
Index: 15264
Index: 15265
Index: 15266
Index: 15267
Index: 15268
Index: 15269
Index: 15270
Index: 15271
Index: 15272
Index: 15273
Index: 15274
Index: 15275
Index: 15276
Index: 15277
Index: 15278
Index: 15279
Index: 15280
Index: 15281
Index: 15282
Index: 15283
Index: 15284
Index: 15285
Index: 15286
Index: 15287
Index: 15288
Index: 15289
Index: 15290
Index: 15291
Index: 15292
Index: 15293
Index: 15294
Index: 15295
Index: 15296
Index: 15297
Index: 15298
Index: 15299
Index: 15300
Index: 15301
Index: 15302
Index: 15303
Index: 15304
Index: 15305
Index: 15306
Index: 15307
Index: 15308
Index: 15309
Index: 15310
Index: 15311
Index: 15312
Index: 15313

In [9]:
####################### Assign Middle and High Schools. #############################################
###### IMPORTANT: Google API may ratelimit, so it may be necessary to kill the process. ######
###### If this happens, export the csv and start from the next index onwards. ################
###### Once ran, do NOT run this code again causing thousands of gmap API calls. #############

# Set parameters to search for primary schools within 10000 meters.
params = {
    "radius": 10000,
    "type": "secondary_school",
    "key": gkey
}

# Construct the baseurl.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in houses_df.iterrows():
    
    # Add lat, lng to the parameters.
    params['location'] = f"{row['latitude']},{row['longitude']}"
    
    # Make request and retrieve the JSON data from the search. 
    #try:
    #    potential_sec_schools = requests.get(base_url, params=params).json()
    #    assigned_sec_schools = get_assigned_secondary_schools(potential_sec_schools, 
    #                                                        (row['latitude'], row['longitude'])
    #                                                        )
        
        # Assign schools.
    #    if assigned_sec_schools[0]:
    #        houses_df.loc[index, 'middle'] = assigned_sec_schools[0]
    #    if assigned_sec_schools[1]:
    #        houses_df.loc[index, 'high'] = assigned_sec_schools[1]
    
    #except Exception as e:
    #    print (f"Index: {index}, Zpid: {row['zpid']}, response message {schools['message']}")
        
    print(f'Index: {index}')

Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Index: 12
Index: 13
Index: 14
Index: 15
Index: 16
Index: 17
Index: 18
Index: 19
Index: 20
Index: 21
Index: 22
Index: 23
Index: 24
Index: 25
Index: 26
Index: 27
Index: 28
Index: 29
Index: 30
Index: 31
Index: 32
Index: 33
Index: 34
Index: 35
Index: 36
Index: 37
Index: 38
Index: 39
Index: 40
Index: 41
Index: 42
Index: 43
Index: 44
Index: 45
Index: 46
Index: 47
Index: 48
Index: 49
Index: 50
Index: 51
Index: 52
Index: 53
Index: 54
Index: 55
Index: 56
Index: 57
Index: 58
Index: 59
Index: 60
Index: 61
Index: 62
Index: 63
Index: 64
Index: 65
Index: 66
Index: 67
Index: 68
Index: 69
Index: 70
Index: 71
Index: 72
Index: 73
Index: 74
Index: 75
Index: 76
Index: 77
Index: 78
Index: 79
Index: 80
Index: 81
Index: 82
Index: 83
Index: 84
Index: 85
Index: 86
Index: 87
Index: 88
Index: 89
Index: 90
Index: 91
Index: 92
Index: 93
Index: 94
Index: 95
Index: 96
Index: 97
Index: 98
Index: 99
Index: 100

Index: 755
Index: 756
Index: 757
Index: 758
Index: 759
Index: 760
Index: 761
Index: 762
Index: 763
Index: 764
Index: 765
Index: 766
Index: 767
Index: 768
Index: 769
Index: 770
Index: 771
Index: 772
Index: 773
Index: 774
Index: 775
Index: 776
Index: 777
Index: 778
Index: 779
Index: 780
Index: 781
Index: 782
Index: 783
Index: 784
Index: 785
Index: 786
Index: 787
Index: 788
Index: 789
Index: 790
Index: 791
Index: 792
Index: 793
Index: 794
Index: 795
Index: 796
Index: 797
Index: 798
Index: 799
Index: 800
Index: 801
Index: 802
Index: 803
Index: 804
Index: 805
Index: 806
Index: 807
Index: 808
Index: 809
Index: 810
Index: 811
Index: 812
Index: 813
Index: 814
Index: 815
Index: 816
Index: 817
Index: 818
Index: 819
Index: 820
Index: 821
Index: 822
Index: 823
Index: 824
Index: 825
Index: 826
Index: 827
Index: 828
Index: 829
Index: 830
Index: 831
Index: 832
Index: 833
Index: 834
Index: 835
Index: 836
Index: 837
Index: 838
Index: 839
Index: 840
Index: 841
Index: 842
Index: 843
Index: 844
Index: 845

Index: 1459
Index: 1460
Index: 1461
Index: 1462
Index: 1463
Index: 1464
Index: 1465
Index: 1466
Index: 1467
Index: 1468
Index: 1469
Index: 1470
Index: 1471
Index: 1472
Index: 1473
Index: 1474
Index: 1475
Index: 1476
Index: 1477
Index: 1478
Index: 1479
Index: 1480
Index: 1481
Index: 1482
Index: 1483
Index: 1484
Index: 1485
Index: 1486
Index: 1487
Index: 1488
Index: 1489
Index: 1490
Index: 1491
Index: 1492
Index: 1493
Index: 1494
Index: 1495
Index: 1496
Index: 1497
Index: 1498
Index: 1499
Index: 1500
Index: 1501
Index: 1502
Index: 1503
Index: 1504
Index: 1505
Index: 1506
Index: 1507
Index: 1508
Index: 1509
Index: 1510
Index: 1511
Index: 1512
Index: 1513
Index: 1514
Index: 1515
Index: 1516
Index: 1517
Index: 1518
Index: 1519
Index: 1520
Index: 1521
Index: 1522
Index: 1523
Index: 1524
Index: 1525
Index: 1526
Index: 1527
Index: 1528
Index: 1529
Index: 1530
Index: 1531
Index: 1532
Index: 1533
Index: 1534
Index: 1535
Index: 1536
Index: 1537
Index: 1538
Index: 1539
Index: 1540
Index: 1541
Inde

Index: 2142
Index: 2143
Index: 2144
Index: 2145
Index: 2146
Index: 2147
Index: 2148
Index: 2149
Index: 2150
Index: 2151
Index: 2152
Index: 2153
Index: 2154
Index: 2155
Index: 2156
Index: 2157
Index: 2158
Index: 2159
Index: 2160
Index: 2161
Index: 2162
Index: 2163
Index: 2164
Index: 2165
Index: 2166
Index: 2167
Index: 2168
Index: 2169
Index: 2170
Index: 2171
Index: 2172
Index: 2173
Index: 2174
Index: 2175
Index: 2176
Index: 2177
Index: 2178
Index: 2179
Index: 2180
Index: 2181
Index: 2182
Index: 2183
Index: 2184
Index: 2185
Index: 2186
Index: 2187
Index: 2188
Index: 2189
Index: 2190
Index: 2191
Index: 2192
Index: 2193
Index: 2194
Index: 2195
Index: 2196
Index: 2197
Index: 2198
Index: 2199
Index: 2200
Index: 2201
Index: 2202
Index: 2203
Index: 2204
Index: 2205
Index: 2206
Index: 2207
Index: 2208
Index: 2209
Index: 2210
Index: 2211
Index: 2212
Index: 2213
Index: 2214
Index: 2215
Index: 2216
Index: 2217
Index: 2218
Index: 2219
Index: 2220
Index: 2221
Index: 2222
Index: 2223
Index: 2224
Inde

Index: 2825
Index: 2826
Index: 2827
Index: 2828
Index: 2829
Index: 2830
Index: 2831
Index: 2832
Index: 2833
Index: 2834
Index: 2835
Index: 2836
Index: 2837
Index: 2838
Index: 2839
Index: 2840
Index: 2841
Index: 2842
Index: 2843
Index: 2844
Index: 2845
Index: 2846
Index: 2847
Index: 2848
Index: 2849
Index: 2850
Index: 2851
Index: 2852
Index: 2853
Index: 2854
Index: 2855
Index: 2856
Index: 2857
Index: 2858
Index: 2859
Index: 2860
Index: 2861
Index: 2862
Index: 2863
Index: 2864
Index: 2865
Index: 2866
Index: 2867
Index: 2868
Index: 2869
Index: 2870
Index: 2871
Index: 2872
Index: 2873
Index: 2874
Index: 2875
Index: 2876
Index: 2877
Index: 2878
Index: 2879
Index: 2880
Index: 2881
Index: 2882
Index: 2883
Index: 2884
Index: 2885
Index: 2886
Index: 2887
Index: 2888
Index: 2889
Index: 2890
Index: 2891
Index: 2892
Index: 2893
Index: 2894
Index: 2895
Index: 2896
Index: 2897
Index: 2898
Index: 2899
Index: 2900
Index: 2901
Index: 2902
Index: 2903
Index: 2904
Index: 2905
Index: 2906
Index: 2907
Inde

...

Index: 12837
Index: 12838
Index: 12839
Index: 12840
Index: 12841
Index: 12842
Index: 12843
Index: 12844
Index: 12845
Index: 12846
Index: 12847
Index: 12848
Index: 12849
Index: 12850
Index: 12851
Index: 12852
Index: 12853
Index: 12854
Index: 12855
Index: 12856
Index: 12857
Index: 12858
Index: 12859
Index: 12860
Index: 12861
Index: 12862
Index: 12863
Index: 12864
Index: 12865
Index: 12866
Index: 12867
Index: 12868
Index: 12869
Index: 12870
Index: 12871
Index: 12872
Index: 12873
Index: 12874
Index: 12875
Index: 12876
Index: 12877
Index: 12878
Index: 12879
Index: 12880
Index: 12881
Index: 12882
Index: 12883
Index: 12884
Index: 12885
Index: 12886
Index: 12887
Index: 12888
Index: 12889
Index: 12890
Index: 12891
Index: 12892
Index: 12893
Index: 12894
Index: 12895
Index: 12896
Index: 12897
Index: 12898
Index: 12899
Index: 12900
Index: 12901
Index: 12902
Index: 12903
Index: 12904
Index: 12905
Index: 12906
Index: 12907
Index: 12908
Index: 12909
Index: 12910
Index: 12911
Index: 12912
Index: 12913

Index: 13468
Index: 13469
Index: 13470
Index: 13471
Index: 13472
Index: 13473
Index: 13474
Index: 13475
Index: 13476
Index: 13477
Index: 13478
Index: 13479
Index: 13480
Index: 13481
Index: 13482
Index: 13483
Index: 13484
Index: 13485
Index: 13486
Index: 13487
Index: 13488
Index: 13489
Index: 13490
Index: 13491
Index: 13492
Index: 13493
Index: 13494
Index: 13495
Index: 13496
Index: 13497
Index: 13498
Index: 13499
Index: 13500
Index: 13501
Index: 13502
Index: 13503
Index: 13504
Index: 13505
Index: 13506
Index: 13507
Index: 13508
Index: 13509
Index: 13510
Index: 13511
Index: 13512
Index: 13513
Index: 13514
Index: 13515
Index: 13516
Index: 13517
Index: 13518
Index: 13519
Index: 13520
Index: 13521
Index: 13522
Index: 13523
Index: 13524
Index: 13525
Index: 13526
Index: 13527
Index: 13528
Index: 13529
Index: 13530
Index: 13531
Index: 13532
Index: 13533
Index: 13534
Index: 13535
Index: 13536
Index: 13537
Index: 13538
Index: 13539
Index: 13540
Index: 13541
Index: 13542
Index: 13543
Index: 13544

Index: 14100
Index: 14101
Index: 14102
Index: 14103
Index: 14104
Index: 14105
Index: 14106
Index: 14107
Index: 14108
Index: 14109
Index: 14110
Index: 14111
Index: 14112
Index: 14113
Index: 14114
Index: 14115
Index: 14116
Index: 14117
Index: 14118
Index: 14119
Index: 14120
Index: 14121
Index: 14122
Index: 14123
Index: 14124
Index: 14125
Index: 14126
Index: 14127
Index: 14128
Index: 14129
Index: 14130
Index: 14131
Index: 14132
Index: 14133
Index: 14134
Index: 14135
Index: 14136
Index: 14137
Index: 14138
Index: 14139
Index: 14140
Index: 14141
Index: 14142
Index: 14143
Index: 14144
Index: 14145
Index: 14146
Index: 14147
Index: 14148
Index: 14149
Index: 14150
Index: 14151
Index: 14152
Index: 14153
Index: 14154
Index: 14155
Index: 14156
Index: 14157
Index: 14158
Index: 14159
Index: 14160
Index: 14161
Index: 14162
Index: 14163
Index: 14164
Index: 14165
Index: 14166
Index: 14167
Index: 14168
Index: 14169
Index: 14170
Index: 14171
Index: 14172
Index: 14173
Index: 14174
Index: 14175
Index: 14176

Index: 14731
Index: 14732
Index: 14733
Index: 14734
Index: 14735
Index: 14736
Index: 14737
Index: 14738
Index: 14739
Index: 14740
Index: 14741
Index: 14742
Index: 14743
Index: 14744
Index: 14745
Index: 14746
Index: 14747
Index: 14748
Index: 14749
Index: 14750
Index: 14751
Index: 14752
Index: 14753
Index: 14754
Index: 14755
Index: 14756
Index: 14757
Index: 14758
Index: 14759
Index: 14760
Index: 14761
Index: 14762
Index: 14763
Index: 14764
Index: 14765
Index: 14766
Index: 14767
Index: 14768
Index: 14769
Index: 14770
Index: 14771
Index: 14772
Index: 14773
Index: 14774
Index: 14775
Index: 14776
Index: 14777
Index: 14778
Index: 14779
Index: 14780
Index: 14781
Index: 14782
Index: 14783
Index: 14784
Index: 14785
Index: 14786
Index: 14787
Index: 14788
Index: 14789
Index: 14790
Index: 14791
Index: 14792
Index: 14793
Index: 14794
Index: 14795
Index: 14796
Index: 14797
Index: 14798
Index: 14799
Index: 14800
Index: 14801
Index: 14802
Index: 14803
Index: 14804
Index: 14805
Index: 14806
Index: 14807

Index: 15362
Index: 15363
Index: 15364
Index: 15365
Index: 15366
Index: 15367
Index: 15368
Index: 15369
Index: 15370
Index: 15371
Index: 15372
Index: 15373
Index: 15374
Index: 15375
Index: 15376
Index: 15377
Index: 15378
Index: 15379
Index: 15380
Index: 15381
Index: 15382
Index: 15383
Index: 15384
Index: 15385
Index: 15386
Index: 15387
Index: 15388
Index: 15389
Index: 15390
Index: 15391
Index: 15392
Index: 15393
Index: 15394
Index: 15395
Index: 15396
Index: 15397
Index: 15398
Index: 15399
Index: 15400
Index: 15401
Index: 15402
Index: 15403
Index: 15404
Index: 15405
Index: 15406
Index: 15407
Index: 15408
Index: 15409
Index: 15410
Index: 15411
Index: 15412
Index: 15413
Index: 15414
Index: 15415
Index: 15416
Index: 15417
Index: 15418
Index: 15419
Index: 15420
Index: 15421
Index: 15422
Index: 15423
Index: 15424
Index: 15425
Index: 15426
Index: 15427
Index: 15428
Index: 15429
Index: 15430
Index: 15431
Index: 15432
Index: 15433
Index: 15434
Index: 15435
Index: 15436
Index: 15437
Index: 15438

In [11]:
############################ Must stay commented out in order not to overwrite. ##############
#houses_df.to_csv('house_info_school_assignments.csv', index=False)
##############################################################################################

In [4]:
# Load data again and remove all addresses with at least one unassigned school. Sava again.
houses_sch_assigned_df = pd.read_csv('house_info_school_assignments.csv')

print(f'There are {len(houses_sch_assigned_df)} records to begin with.')

houses_sch_assigned_df['elem'].replace('', np.nan, inplace=True)
houses_sch_assigned_df['middle'].replace('', np.nan, inplace=True)
houses_sch_assigned_df['high'].replace('', np.nan, inplace=True)
houses_sch_assigned_df.dropna(subset=['elem',
                                      'middle',
                                      'high'], 
                                      inplace=True)

print(f'There are {len(houses_sch_assigned_df)} records after dropping houses with at least one unassigned school.')

houses_sch_assigned_df.to_csv('house_info_school_assignments_dropna.csv', index=False)

There are 15818 records to begin with.
There are 13955 records after dropping houses with at least one unassigned school.
